In [2]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator
from Data_setup import data_set

fig = go.Figure()


dataset = data_set

for i in dataset:
    headerlength = get_header_length(i['data_CV'])
    data = pd.read_csv(i['data_CV'], encoding='ansi', sep="\t", decimal=get_decimal_separator(i['data_CV']),  skiprows=range(headerlength))
    cn = data['cycle number']
    data = data[data['cycle number']== float(cn[-1:])] # allowing this comment will plot only the last scan. If you want to plot all of the scans then comment this out. 
    voltage = data['Ewe/V']
    current = data['<I>/mA']
    current = current / i['mass']
    voltage_corrected = data['Ewe/V'] - data['<I>/mA'] * i['Ru']/1000 - i['cal'] #does the iR correction converts to overpotential

    fig.add_trace(go.Scatter(
        x=voltage_corrected,
        y=current,
        name= i['label']
    ))
fig.update_xaxes(range=[0.00, 0.750],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[-5, 120],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Cyclic Voltammetry.jpeg')
fig.write_image('Cyclic Voltammetry.svg')

